In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics
from sklearn.model_selection import cross_val_score

# Load Data
data = pd.read_csv('/Users/sauce/Desktop/DraftKings/ready_data/DraftKingsCleaned.csv', header = None)

# rename columns
data = data.rename(columns={0: "season", 
                        1:'game_date',
                        2: 'player',
                        3: 'position',    
                        4: 'team',
                        5: 'opponent',
                        6: 'venue',
                        7: 'minutes',
                        8: 'usage_rate',
                        9: 'rest',
                        10: 'avg_threes',
                        11: 'avg_reb',
                        12: 'avg_ast',
                        13: 'avg_stl',
                        14: 'avg_blk',
                        15: 'avg_tov',
                        16: 'avg_pts',
                        17: 'avg_points_vs_opp',
                        18: 'team_pace',
                        19: 'team_ast',
                        20: 'team_tov',
                        21: 'team_reb_rate',
                        22: 'team_offeff',
                        23: 'team_defeff',
                        24: 'opp_pace',
                        25: 'opp_ast',
                        26: 'opp_tov',
                        27: 'opp_reb_rate',
                        28: 'opp_offeff',
                        29: 'opp_defeff',
                        30: 'opp_pos_avg',
                        31: 'salary',
                        32: 'fantasy_points',
                       })

# Avg 10 data
data["game_date"] = pd.to_datetime(data.game_date)
data.set_index('game_date', inplace=True)
data.sort_index(inplace=True)
df_rolling = data.groupby(['player']).rolling(10).mean().rename(columns={'season':'season1', 'player':'player1'}).reset_index()
data = data.reset_index()
df_rolling = df_rolling.drop(columns=['player'])
df_rolling = df_rolling.rename(columns = {'season1': 'season', 'player1': 'player'})
data = pd.merge(data, df_rolling, on=['player', 'season', 'game_date'], left_index= True , suffixes=['', '_AVG10'])

# Avg 3 data
data["game_date"] = pd.to_datetime(data.game_date)
data.set_index('game_date', inplace=True)
data.sort_index(inplace=True)
df_rolling = data.groupby(['player']).rolling(3).mean().rename(columns={'season':'season1', 'player':'player1'}).reset_index()
data = data.reset_index()
df_rolling = df_rolling.drop(columns=['player'])
df_rolling = df_rolling.rename(columns = {'season1': 'season', 'player1': 'player'})
df = pd.merge(data, df_rolling, on=['player', 'season', 'game_date'], left_index= True , suffixes=['', '_AVG3'])

# Drop useless features
df = df.drop(columns=['team_AVG10','opponent_AVG10','venue_AVG10','rest_AVG10',
                      'avg_threes_AVG10','avg_reb_AVG10','avg_ast_AVG10','avg_stl_AVG10',
                      'avg_blk_AVG10','avg_tov_AVG10','avg_pts_AVG10','avg_points_vs_opp_AVG10',
                      'team_pace_AVG10','team_ast_AVG10','team_tov_AVG10','team_reb_rate_AVG10',
                      'team_defeff_AVG10','opp_pace_AVG10','opp_ast_AVG10','opp_tov_AVG10',
                      'opp_reb_rate_AVG10','opp_offeff_AVG10','salary_AVG10',
                      'team_AVG3', 'opponent_AVG3', 'venue_AVG3','rest_AVG3',
                      'avg_threes_AVG3', 'avg_reb_AVG3','avg_ast_AVG3', 'avg_stl_AVG3', 
                      'avg_blk_AVG3', 'avg_tov_AVG3','avg_pts_AVG3', 
                      'avg_points_vs_opp_AVG3', 'team_pace_AVG3','team_ast_AVG3', 
                      'team_tov_AVG3', 'team_reb_rate_AVG3','team_defeff_AVG3', 
                      'opp_pace_AVG3','opp_ast_AVG3', 'opp_tov_AVG3', 'opp_reb_rate_AVG3',
                      'opp_offeff_AVG3', 'opp_pos_avg_AVG3','salary_AVG3',
                      'team_AVG10_AVG3','opponent_AVG10_AVG3', 'venue_AVG10_AVG3', 
                      'minutes_AVG10_AVG3','usage_rate_AVG10_AVG3', 'rest_AVG10_AVG3',
                      'avg_threes_AVG10_AVG3', 'avg_reb_AVG10_AVG3',
                      'avg_ast_AVG10_AVG3', 'avg_stl_AVG10_AVG3', 'avg_blk_AVG10_AVG3',
                      'avg_tov_AVG10_AVG3', 'avg_pts_AVG10_AVG3',
                      'avg_points_vs_opp_AVG10_AVG3', 'team_pace_AVG10_AVG3',
                      'team_ast_AVG10_AVG3', 'team_tov_AVG10_AVG3',
                      'team_reb_rate_AVG10_AVG3', 'team_offeff_AVG10_AVG3',
                      'team_defeff_AVG10_AVG3', 'opp_pace_AVG10_AVG3',
                      'opp_ast_AVG10_AVG3', 'opp_tov_AVG10_AVG3',
                      'opp_reb_rate_AVG10_AVG3', 'opp_offeff_AVG10_AVG3',
                      'opp_defeff_AVG10_AVG3', 'opp_pos_avg_AVG10_AVG3',
                      'salary_AVG10_AVG3', 'fantasy_points_AVG10_AVG3', 'position_AVG10_AVG3', 'position_AVG3', 'position_AVG10'])

# Fill NAs
df['fantasy_points_AVG3'] = df['fantasy_points_AVG3'].fillna(df.groupby(['player', 'season'])['fantasy_points'].transform('mean'))
df['opp_defeff_AVG3'] = df['opp_defeff_AVG3'].fillna(df.groupby(['opponent', 'season'])['opp_defeff'].transform('mean'))
df['team_offeff_AVG3'] = df['team_offeff_AVG3'].fillna(df.groupby(['team', 'season'])['team_offeff'].transform('mean'))
df['usage_rate_AVG3'] = df['usage_rate_AVG3'].fillna(df.groupby(['player', 'season'])['usage_rate'].transform('mean'))
df['fantasy_points_AVG10'] = df['fantasy_points_AVG10'].fillna(df.groupby(['player', 'season'])['fantasy_points'].transform('mean'))
df['minutes_AVG3'] = df['minutes_AVG3'].fillna(df.groupby(['player', 'season'])['minutes'].transform('mean'))
df['opp_pos_avg_AVG10'] = df['opp_pos_avg_AVG10'].fillna(df.groupby(['opponent', 'season'])['opp_pos_avg'].transform('mean'))
df['opp_defeff_AVG10'] = df['opp_defeff_AVG10'].fillna(df.groupby(['opponent', 'season'])['opp_defeff'].transform('mean'))
df['team_offeff_AVG10'] = df['team_offeff_AVG10'].fillna(df.groupby(['team', 'season'])['team_offeff'].transform('mean'))
df['usage_rate_AVG10'] = df['usage_rate_AVG10'].fillna(df.groupby(['player', 'season'])['usage_rate'].transform('mean'))
df['minutes_AVG10'] = df['minutes_AVG10'].fillna(df.groupby(['player', 'season'])['minutes'].transform('mean'))

# # Drop non predictive columns 
# df2 = df.drop(columns=['game_date', 'season', 'player', 'team', 'opponent',
#                      'minutes', 'usage_rate', 'salary', 'position'])
# # Encode Dummies
# df2 = pd.get_dummies(df2, columns=['venue', 'rest'], drop_first=True)

# # Grab Target Variable and remove it from data.
# y = df2['fantasy_points']
# X = df2.drop(columns = ['fantasy_points'])

# # Split data into train and test sets
# # from sklearn.cross_validation import train_test_split
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# # Scale data
# # from sklearn.preprocessing import StandardScaler
# # sc_X = StandardScaler()
# # X_train = sc_X.fit_transform(X_train)
# # X_test = sc_X.transform(X_test)

In [22]:
df.shape

(47738, 44)

In [ ]:
import statsmodels.formula.api as sm
X = np.append(arr = np.ones((45576, 1)).astype(int), values = X, axis = 1)

X_opt = X[:, [1,2,3,4,5,7,8,
             13,14,19,20,
             21,22,26,27,28,30,
             31,32,33,36]]
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()

In [16]:
from sklearn.cross_validation import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size = 0.2, random_state = 0)

from sklearn.linear_model import LinearRegression
Mregressor2 = LinearRegression(normalize=True)
Mregressor2.fit(X_train2, y_train2)

y_pred2 = Mregressor2.predict(X_test2)

from sklearn import metrics
print('Mean Squared Error:', metrics.mean_squared_error(y_test2, y_pred2))
print('R2 Score:', metrics.r2_score(y_test2, y_pred2))

Mean Squared Error: 34.303077046694625
R2 Score: 0.8309063763918545


In [14]:
X.columns.values

array(['position', 'avg_threes', 'avg_reb', 'avg_ast', 'avg_stl',
       'avg_blk', 'avg_tov', 'avg_pts', 'avg_points_vs_opp', 'team_pace',
       'team_ast', 'team_tov', 'team_reb_rate', 'team_offeff',
       'team_defeff', 'opp_pace', 'opp_ast', 'opp_tov', 'opp_reb_rate',
       'opp_offeff', 'opp_defeff', 'opp_pos_avg', 'minutes_AVG10',
       'usage_rate_AVG10', 'team_offeff_AVG10', 'opp_defeff_AVG10',
       'opp_pos_avg_AVG10', 'fantasy_points_AVG10', 'minutes_AVG3',
       'usage_rate_AVG3', 'team_offeff_AVG3', 'opp_defeff_AVG3',
       'fantasy_points_AVG3', 'venue_R', 'rest_2', 'rest_3+', 'rest_3IN4',
       'rest_3IN4-B2B', 'rest_4IN5', 'rest_4IN5-B2B', 'rest_5IN5-B2B2B',
       'rest_B2B'], dtype=object)

In [5]:
y_pred2

array([22.69037158,  1.83813215, 29.20377612, ..., 23.97730135,
        1.99931182, 26.98127948])